In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import asyncio
import json

from loguru import logger


loop = asyncio.get_running_loop()

In [ ]:
with open('settings/testnet.json', 'r') as f:
    tonlib_config = json.load(f)
    
tonlib_config['liteservers'] = tonlib_config['liteservers']
print('Num ls:', len(tonlib_config['liteservers']))

In [ ]:
os.makedirs('private/ton_keystore', exist_ok=True)

# TonlibMulticlient

In [ ]:
loop = asyncio.get_running_loop()

In [ ]:
from tApi.tonlib.multiclient import TonlibMultiClient



liteserver_blacklist = [3, 6, 13]
multiclient = TonlibMultiClient(loop, 
                                tonlib_config, 
                                './private/ton_keystore',
                                liteserver_blacklist=liteserver_blacklist,
                                verbosity_level=4)
await multiclient.init(0)

In [ ]:
await multiclient.getMasterchainInfo()

In [ ]:
from ipywidgets import Output

output = Output()
async def monitor():
    while True:
        res = ''
        for client in multiclient.clients.values():
            res += f'{client.ls_index:2d}: {len(client.tonlib_wrapper.futures):3d}\n'
        with output:
            output.clear_output(wait=True)
            print(res)
        await asyncio.sleep(0.25)
        
display(output)
asyncio.ensure_future(monitor())

In [ ]:
import time
from tqdm.auto import tqdm


for i in tqdm(range(16384)):
    fut = multiclient.getBlockTransactions(-1, -9223372036854775808, 2000, 10)
    asyncio.ensure_future(fut)
    await asyncio.sleep(0.0005)

In [ ]:
[3, 6, 13]

# TonlibClient

In [ ]:
loop = asyncio.get_running_loop()

In [ ]:
from tApi.tonlib.client import TonlibClient


client = TonlibClient(1, 
                      tonlib_config, 
                      './private/ton_keystore',
                      loop,
                      cdll_path=None,
                      verbosity_level=0)
await client.init()

In [ ]:
masterchain_info = await client.getMasterchainInfo()
last_block = masterchain_info['last'].copy()
last_block.pop('@type')

block_header = await client.getBlockHeader(**last_block)

transactions = await client.getBlockTransactions(**last_block, count=50)
transactions